In [1]:
from google.cloud import storage

path_to_private_key = 'is3107-418903-a6116d294003.json'

storage_client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

In [2]:
from google.api_core.exceptions import Forbidden, NotFound

def download_blob(bucket_name,source_blob_name, destination_file_name):
    try:
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)
    except Forbidden as e:
        print(f"Access denied. You might have a billing or permissions issue: {e}")
    except NotFound as e:
        print(f"The requested file was not found: {e}")

    

# download model
download_blob('is3107-model', 'ml_model/modelRF.pkl', 'modelRF.pkl')

# download model dataset
download_blob('is3107-model', 'ml_model/dataset.csv', 'dataset.csv')

In [3]:
%pip install -U scikit-learn==1.3.2

Note: you may need to restart the kernel to use updated packages.


In [4]:
import joblib
import pandas as pd

modelRF = joblib.load('modelRF.pkl')
data = pd.read_csv('dataset.csv')

In [5]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

y = data['price']
x = data.drop('price', axis=1)

grid_searchRF = GridSearchCV(modelRF, param_grid, cv=3, scoring='accuracy')

# Fit on your data
grid_searchRF.fit(x, y)


# Best model
best_modelRF = grid_searchRF.best_estimator_

/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/isabel/opt/anaconda3/envs/IS3107/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  

# Predict on user input

In [6]:
data_dict = {
    'owners': 2,
    'eng_cap': 1580.0,
    'depreciation': 16954.0,
    'mileage': 111000.0,
    'power': 1036.0,
    'coe_left': 3.2,
    'omv': 27383.0,
    'arf': 5000.0,
    'model_seats': 5,
    'model_weight_kg': 1433.560091,
    'model_engine_cyl': 4,
    'model_fuel_cap_l': 45.0415,
    'years_since_reg': 7,
    'model_body_Coupe': False,
    'model_body_Hatchback': False,
    'model_body_Minivan': False,
    'model_body_SUV': True,
    'model_body_Sedan': False,
    'model_transmission_type_CVT': False,
    'model_transmission_type_Manual': True,
    'model_drive_Front Wheel Drive': True,
    'model_drive_Rear Wheel Drive': False,
    'model_engine_fuel_Premium': False,
    'model_engine_fuel_Regular': True,
    'brands': 1.0
}

In [7]:
data_df = pd.DataFrame([data_dict])

In [8]:
predictions = best_modelRF.predict(data_df)

In [9]:
predictions[0]

60783.65378189093